In [40]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [41]:
import numpy as np
import tensorflow as tf
from utils_functions import *
from plan import *
import random
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorboard.plugins.hparams import api as hp
import datetime, os

In [42]:
# controllo di star effettivamente usando la GPU

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))

Num GPUs Available:  1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [43]:
dict_stati= load_file("./dizionario_stati")
plansLoaded=load_file("./plans")

File loaded
File loaded


In [44]:
#ogni stato è un tensore con elementi dtype int8, questi stati vengono raggruppati in un altro tensore che rappresnta la
#variabile "states" del piano 

def build_vector(dict, states_list):
    l=len(dict)
    vector_states=[]
    for state in states_list:
        vector=np.array([0]*l,dtype=np.int8)
        #vector=tf.zeros(l,dtype=np.int8)
        for s in state:
            for key in dict.keys():
                if key==s:
                    vector[dict[key]-1]=1
                    break
        #t=tf.convert_to_tensor(vector,dtype=tf.int8)            
        vector_states.append(vector) 
    r = np.array(vector_states)                          
    return r

#NON UTILIZZATO ADESSO
#shape di ogni singolo elemento (r) è (n x 340) con n che varia su ogni piano, raggruppati sulla base dei piani

def build_all_vectors(dict,plans_list):
    total=[]
    for plan in plans_list:
        r=build_vector(dict,plan.states)
        total.append(r)
    #r = tf.convert_to_tensor(total,dtype=None)                         
    return total
 
    
#gli stati vengono ordinati (in una lista) singolarmente con shape (1x340), non vengono raggruppati sulla base dei piani
#da utilizzare per autoencoder standard

def build_all_vectors1x340(dict, plans_list):
    l=len(dict)
    total=[]
    for plan in plans_list:
        for state in plan.states:
            vector=np.array([0]*l,dtype=np.int8)
            for s in state:
                for key in dict.keys():
                    if key==s:
                        vector[dict[key]-1]=1
                        break
            #t=tf.convert_to_tensor(vector,dtype=tf.int8)            
            total.append(vector)
    total=np.array(total)        
    return total        

## Preparazione dati

### Caricamento dataset statici

In [45]:
test=load_file("./Dataset/set_test")
train=load_file("./Dataset/set_training")
validation=load_file("./Dataset/set_validation")

File loaded
File loaded
File loaded


In [46]:
test=np.array(test,dtype=np.int8)
train=np.array(train,dtype=np.int8)
validation=np.array(validation,dtype=np.int8)

In [47]:
logdir = os.path.join("./TestLogs/", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=10),
    tf.keras.callbacks.TensorBoard(log_dir=logdir,histogram_freq=1, write_graph=True, write_images=True)
]

In [48]:
# Per il test mi limito a tunare il numero di unità nascoste.
# Se poi funziona, dovrebbe trattarsi solo di scrivere meccanicamente le righe restanti.
input_size = 340
HP_HIDDEN_SIZE = hp.HParam('hidden_size', hp.Discrete([160, 180]))
code_size = 85
METRIC_LOSS = 'loss'

with tf.summary.create_file_writer('./logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_HIDDEN_SIZE],
        metrics=[hp.Metric(METRIC_LOSS, display_name='loss')],
    )

In [49]:
# def train_model(hparams):
#     input_layer = Input(shape=(input_size,))
#     hidden_1 = Dense(hparams[HP_HIDDEN_SIZE], activation='swish', kernel_initializer="he_uniform", )(input_layer)
#     code = Dense(code_size, activation='swish',kernel_initializer="he_uniform",)(hidden_1)
#     hidden_2 =Dense(hparams[HP_HIDDEN_SIZE], activation='swish',kernel_initializer="he_uniform",)(code)
#     output_layer = Dense(input_size, activation='sigmoid')(hidden_2)
    
#     autoencoder = Model(input_layer, output_layer)
#     autoencoder.compile(
#         optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#         loss='binary_crossentropy',
#         metrics=['Precision'],
#     )
#     autoencoder.fit(
#         x=train,y=train,
#         epochs=15,
#         batch_size=1000,
#         validation_data=(validation,validation),
#         callbacks=my_callbacks,
#     )
    
#     _, Precision = model.evaluate(x=test, y=test)
#     return Precision

def train_model(hparams):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(input_size,)),
        tf.keras.layers.Dense(hparams[HP_HIDDEN_SIZE], activation=tf.nn.swish),
        tf.keras.layers.Dense(code_size, activation=tf.nn.swish),
        tf.keras.layers.Dense(hparams[HP_HIDDEN_SIZE], activation=tf.nn.swish),
        tf.keras.layers.Dense(input_size, activation=tf.nn.sigmoid),
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['loss'],
    )
    
    model.fit(
        x=train,y=train,
        epochs=1500,
        batch_size=10000,
        validation_data=(validation,validation),
        callbacks=my_callbacks,
    )
    _, epoch_loss = model.evaluate(x=test, y=test)
    return epoch_loss
    

In [50]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)
        loss = train_model(hparams)
        tf.summary.scalar(METRIC_LOSS, loss, step=1)

In [51]:
session_num = 0

for h_units in HP_HIDDEN_SIZE.domain.values:
    hparams= {
        HP_HIDDEN_SIZE: h_units,
    }
    run_name = "run-%d" % session_num
    print('--- Starting trial: %s' % run_name)
    print({h.name: hparams[h] for h in hparams})
    run('logs/hparam_tuning/' + run_name, hparams)
    session_num += 1

--- Starting trial: run-0
{'hidden_size': 160}
Epoch 1/1500


ValueError: in user code:

    C:\Users\lucal\miniconda3\envs\EnvAutoEncoder\lib\site-packages\tensorflow\python\keras\engine\training.py:862 train_function  *
        return step_function(self, iterator)
    C:\Users\lucal\miniconda3\envs\EnvAutoEncoder\lib\site-packages\tensorflow\python\keras\engine\training.py:852 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\lucal\miniconda3\envs\EnvAutoEncoder\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\lucal\miniconda3\envs\EnvAutoEncoder\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\lucal\miniconda3\envs\EnvAutoEncoder\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
   

In [ ]:
%tensorboard --logdir ./logs/hparam_tuning